In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from scipy.spatial.transform import Rotation as R

In [100]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

In [3]:
from src.data.ShapeManager import Data
data = Data(np.load('./data/test16.npy'), np.load('./data/input_norm.npy'), "input")

In [92]:
def get_env_coords():
    diameter = size / float((res - 1))
    coverage = 0.5 * diameter
    points = []
    for y in range(layers): # height
        for z in range(res): # radius
            dist = z * coverage
            arc = 2 * np.pi * dist
            count = int(np.rint(arc / coverage))
            for x in range(count): # angle
                degrees = x / count * 2 * np.pi
                coords = [dist * np.cos(degrees), y * coverage, dist * np.sin(degrees)]
                    points.append(coords)
    return np.array(points)

def draw_points(points = None, env = None, ax = None):
    if ax is None:
        fig = plt.figure()
        ax = fig.add_subplot(111,projection='3d')
    
    if env is not None:
        ax.set_xlim3d(env.bounds[0][0], env.bounds[0][0] + env.bounds[0][1])
        ax.set_ylim3d(env.bounds[1][0], env.bounds[1][0] + env.bounds[1][1])
        ax.set_zlim3d(env.bounds[2][0], env.bounds[2][0] + env.bounds[2][1])

    X,Z,Y = points.T

    ax.scatter(X,Y,Z, alpha = 0.5)
    return ax

def draw_env(env):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    ax.set_xlim3d(env.bounds[0][0], env.bounds[0][0] + env.bounds[0][1])
    ax.set_ylim3d(env.bounds[1][0], env.bounds[1][0] + env.bounds[1][1])
    ax.set_zlim3d(env.bounds[2][0], env.bounds[2][0] + env.bounds[2][1])
    
    points = np.array(np.where(env.map > 0)).T
    points = points / env.res * env.bounds[:,1] + env.bounds[:,0]
    return draw_points(points, ax = ax)
    
    
def draw_obj(obj, position = [0,0,0], direction = [1,0,0], ax = None):
    points = obj.get(position, direction)
    points = points[:,:3]
    return draw_points(points, ax = ax)

def draw_collider(X):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    env_points = get_env_coords()
    points = env_points[X > 0]
    return draw_points(points, ax = ax)
        
class Environment(object):
    def __init__(self, bounds, res):
        self.bounds = np.array(bounds)
        self.bounds[:,1] -= self.bounds[:,0]
        self.res = res
        self.map = np.zeros((self.res,self.res,self.res))
        
        diameter = size / float((9 - 1))
        self.fill_points = self.get_sphere(diameter)
    
    def get_sphere(self, radius):
        rx,ry,rz = np.int32([radius,radius,radius] / self.bounds[:,1] * self.res)
        points = np.array(np.meshgrid(range(-rx,rx), range(-ry,ry), range(-rz,rz))).reshape(3,-1).T
        radii = np.linalg.norm(points / [rx,ry,rz], axis = 1)
        points =  points[radii <= radius]
        return np.int32(points)

    def grid(self, coords, value = None, radius = None):
        coords = (np.array(coords) - self.bounds[:,0]) / self.bounds[:,1] * self.res
        x,y,z = np.int32(coords)
        if value is not None:
            if radius != None:
                indices = tuple((self.fill_points + [x,y,z]).T)
                self.map[indices] = value
            else:
                self.map[x,y,z] = value
        return self.map[x,y,z]

    def set_map(self, points, position = [0,0], direction = [1,0]):
        position, direction = np.array(position), np.array(direction)
        self.map = np.zeros((self.res,self.res,self.res))
        
        diameter = size / float((res - 1))
        coverage = 0.5 * diameter
        idx = 0
        
        for y in range(layers): # height
            for z in range(res): # radius
                dist = z * coverage
                arc = 2 * np.pi * dist
                count = int(np.rint(arc / coverage))

                for x in range(count): # angle
                    degrees = x / count * 2 * np.pi
                    coords = [dist * np.cos(degrees), y * coverage, dist * np.sin(degrees)]
                    self.grid(coords, points[idx], diameter)
                    idx += 1

    def get(self, position = np.array([0,0]), direction = np.array([1,0])):
        position,direction = np.array(position), np.array(direction)
        diameter = size / float((res - 1))
        coverage = 0.5 * diameter
        points = []
        _x,_y = position
        _w = np.arctan2(direction[1], direction[0])
        
        for y in range(layers): # height
            for z in range(res): # radius
                dist = z * coverage
                arc = 2 * np.pi * dist
                count = int(np.rint(arc / coverage))
                for x in range(count): # angle
                    degrees = x / count * 2 * np.pi
                    coords = [_x + dist * np.cos(_w + degrees), y * coverage, _y + dist * np.sin(_w + degrees)]

                    points.append(self.grid(coords))
        return np.array(points)

class World(object):
    def __init__(self, environment, objects):
        self.environment = environment
        self.objects = objects

    def get(self, position, object_idx = None):
        obj = None if object_idx is None else self.objects[object_idx]
        obj = None if obj is None else obj.get(position)
        env = self.environment.get(position)
        
        return env, obj

In [93]:
class Object(object):
    def __init__(self, obj, position, direction):
        self.points = obj[:,:3]
        self.occ = obj[:,3]
        self.position = position
        self.direction = direction
        self.up = np.array([0,1,0])
        
    def get(self, position, direction):
        points = self.points @ rotation_matrix(direction, self.up).T + position
        return np.concatenate([points, self.occ[:,None]],1)
        
        
def rotation_matrix(forward, up):
    forward, up = np.float32(forward) / np.linalg.norm(forward), np.float32(up) / np.linalg.norm(forward)
    left = np.cross(forward, up)
    up = np.cross(left, forward)
    
    return np.array([forward, up, left])

In [94]:
size, res, layers = 4,9,9
env_size = 4
bounds = [[-env_size, env_size],[-env_size, env_size],[-env_size, env_size]]

sample = data.environment[-1]

env = Environment(bounds, 80)
env.map[:,40:42,:] = 1

sample = data.interaction[2175].reshape(-1,4)
obj = Object(sample, [0,0,0], [1,0,0])

In [96]:
draw_collider(env.get())
env.get()

draw_points(get_env_coords())
ax = draw_env(env)
draw_obj(obj, ax = ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [97]:
import numpy as np

class Transform:
    def __init__(self, matrix):
        self.matrix = np.array(matrix)
    
    def rotation(self):
        return self.matrix[:3,:3]
    
    def rotation_gl(self):
        return [0,0]
    
    def translation(self):
        return self.matrix[:3,3]
    
    def apply(self, points):
        points = np.array(points)
        if points.ndim == 1: points = points[None]
        points = np.concatenate([points, np.ones([len(points),1])], 1)
        return np.squeeze(points @ self.matrix.T)
    
    def inverse(self):
        M = self.matrix.copy()
        M[:3,:3] = self.rotation().T
        M[:3,3] = - self.rotation().T @ self.translation()
        return Transform(M)
    
    def local(self, point):
        return point - self.translation()
    
    def mul(self, other):
        return Transform(self.matrix @ other.matrix)
    
    def __repr__(self):
        return np.array2string(self.matrix, precision = 4, suppress_small=True)
    
def from_euler(angles):
    mat = np.eye(4)
    mat[:3,:3] = Rotation.from_euler('xyz', angles).as_matrix()
    return Transform(mat)

def from_vec(vec):
    mat = np.eye(4)
    mat[:3,3] = vec
    return Transform(mat)

class Box:
    def __init__(self, transform, size):
        self.transorm = transform
        self.size = size
    
    def collide(self, points):
        return None
    
class BoxEnv:
    def __init__(self, boxes, sensor_radius = 4, layers = 9, res = 9):
        self.boxes = boxes
        self.size = sensor_radius
        self.layers = layers
        self.res = res
        
    def collide_point(self, point):
        return (self.size - np.minimum(np.linalg.norm(np.maximum((p - X) * [[-1],[1]], 0), axis = 2).max(1), self.size)).min()
    
    def collider_points(self, position = [0,0], direction = [1,0]):
        position,direction = np.array(position), np.array(direction)
        diameter = self.size / float((self.res - 1))
        coverage = 0.5 * diameter
        points = []
        _x,_y = position
        _w = np.arctan2(direction[1], direction[0])
        
        for y in range(self.layers): # height
            for z in range(self.res): # radius
                dist = z * coverage
                arc = 2 * np.pi * dist
                count = int(np.rint(arc / coverage))
                for x in range(count): # angle
                    degrees = x / count * 2 * np.pi
                    coords = [_x + dist * np.cos(_w + degrees), y * coverage, _y + dist * np.sin(_w + degrees)]
                    points.append(coords)
        return np.array(points)
    
    

In [98]:
env = BoxEnv([])

In [99]:
draw_points(env.collider_points())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …